In [ ]:
#pip install trino urllib3 pandas

In [ ]:
# sqlalchemy-trino has an integration bug w/ trino api, possibly fixed but unreleased
# also currently requires sqlalchemy 1.3, not 1.4
# however if they get this fixed it might be another useful connection api
#pip install --upgrade sqlalchemy==1.3 sqlalchemy-trino

In [ ]:
# This may not be compatible with installing sqlalchemy-trino
#pip install 'pyhive[presto,trino]'
# https://github.com/dropbox/PyHive

In [ ]:
# two possible apis to generate a trino connection:
import trino
from pyhive import presto

# pandas dfs
import pandas as pd

import urllib3
urllib3.disable_warnings()

In [ ]:
from dotenv import dotenv_values, load_dotenv
import os
import pathlib

dotenv_dir = os.environ.get('CREDENTIAL_DOTENV_DIR', os.environ.get('PWD', '/opt/app-root/src'))
dotenv_path = pathlib.Path(dotenv_dir) / 'credentials.env'
if os.path.exists(dotenv_path):
    load_dotenv(dotenv_path=dotenv_path,override=True)

In [ ]:
JWT_TOKEN = os.environ['TRINO_PASSWD']
conn = trino.dbapi.connect(
    host=os.environ['TRINO_HOST'],
    port=int(os.environ['TRINO_PORT']),
    user=os.environ['TRINO_USER'],
    http_scheme='https',
    auth=trino.auth.JWTAuthentication(JWT_TOKEN),
    verify=True,
)
cur = conn.cursor()

In [ ]:
cur.execute('show catalogs')
cur.fetchall()

In [ ]:
# Show available schemas to ensure trino connection is set correctly
cur.execute('show schemas in osc_datacommons_dev')
cur.fetchall()

In [ ]:
cur.execute('show tables from osc_datacommons_dev.team1')
cur.fetchall()

In [ ]:
# this doesn't work with connection from trino api
# but it will work with connection from pyhive.presto, see below
#catDF = pd.read_sql("select * from hive.team1.cat", conn)

In [ ]:
cur.execute('select * from osc_datacommons_dev.team1.cat')
df = pd.DataFrame(cur.fetchall())

# hack to move column names out of data
df, df.columns = df[1:], df.iloc[0]
df.reset_index(drop=True)

# infer data types (these will be str)
df = df.convert_dtypes()

# convert length to int
df['length'] = pd.to_numeric(df.length)

df.head()

In [ ]:
# check column types
df.dtypes

## Using a pyhive.presto connection

In [ ]:
import pyhive.presto
JWT_TOKEN = os.environ['TRINO_PASSWD']
conn = presto.connect(
    username = os.environ['TRINO_USER'],
    password = ???,
    host = os.environ['TRINO_HOST'],
    port = int(os.environ['TRINO_PORT']),
    protocol = 'https'
)

In [ ]:
# With pyhive.presto you can load directly from the connection,
# although resulting DF is same as from trino api
df = pd.read_sql("select * from osc_datacommons_dev.team1.cat", conn)
df.head()

## **** Below is not working
see:
https://github.com/dungdm93/sqlalchemy-trino/issues/23

In [ ]:
sqlstring = 'trino://{user}:{passwd}@{host}:{port}/hive'.format(
    user = trino_user,
    passwd = trino_passwd,
    host=trino_host,
    port=trino_port
)
sqlstring

In [ ]:
import sqlalchemy
engine = sqlalchemy.engine.create_engine(
    sqlstring)

In [ ]:
df = pd.read_sql("select * from hive.team1.cat", engine)

In [ ]:
import sqlalchemy

sqlstring = 'trino://{user}:{passwd}@{host}:{port}/hive'.format( \
    user = trino_user, \
    passwd = trino_passwd, \
    host=trino_host, \
    port=trino_port
)
engine = sqlalchemy.create_engine(sqlstring)

In [ ]:
# https://github.com/dungdm93/sqlalchemy-trino/issues/23
pd.read_sql_query('select * from hive.team1.cat', sqlstring)